In [2]:
import os
import sys
sys.path.append( os.path.abspath("../../workspace/python-work/python 3/cs230_deeplearning_implement/ec_code/")  )
from class5.week3.MachineTranslation.nmt_utils import *

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt

%matplotlib inline

#构建一个神经机器翻译 (NMT) 模型，将人类可读日期 (“25th of June, 2009”) 翻译为机器可读日期 (“2009-06-25”).
#网络将输入以各种可能格式编写的日期 (例如：“the 29th of August 1958”, “03/30/1968”, “24 JUNE 1987”) 将它们转换为标准化的机器可读日期 (例如：“1958-08-29”, “1968-03-30”, “1987-06-24”).
# 我们将让网络学会以通用的机器可读格式输出日期YYYY-MM-DD.
# 使用注意模型执行此操作, 序列模型中最复杂的序列之一。


Using TensorFlow backend.


In [3]:
m = 10000

#dataset: 一个元组列表 (人类可读日期, 机器可读日期)。
# human_vocab: 一个字典，将人类可读日期中使用的所有字符映射到整数值索引。
# machine_vocab: 一个字典，将机器可读日期中使用的所有字符映射到整数值索引。这些索引不一定与 human_vocab 的索引一致。
# inv_machine_vocab: machine_vocab的逆字典，从索引到字符的映射。
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

dataset[:10]



  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]

  9%|██████▊                                                                     | 899/10000 [00:00<00:01, 8906.11it/s]

 21%|████████████████                                                           | 2142/10000 [00:00<00:00, 9712.51it/s]

 36%|██████████████████████████▎                                               | 3557/10000 [00:00<00:00, 10698.26it/s]

 44%|████████████████████████████████▊                                          | 4368/10000 [00:00<00:00, 7750.75it/s]

...

 77%|█████████████████████████████████████████████████████████▌                 | 7676/10000 [00:00<00:00, 6913.30it/s]

 84%|███████████████████████████████████████████████████████████████            | 8402/10000 [00:01<00:00, 6994.68it/s]

 91%|████████████████████████████████████████████████████████████████████▎      | 9113/10000 [00:01<00:00, 7009.30it/s]

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 8401.15it/s]

[('9 may 1998', '1998-05-09'),
 ('10.09.70', '1970-09-10'),
 ('4/28/90', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('sunday may 22 1988', '1988-05-22'),
 ('tuesday july 8 2008', '2008-07-08'),
 ('08 sep 1999', '1999-09-08'),
 ('1 jan 1981', '1981-01-01'),
 ('monday may 22 1995', '1995-05-22')]

In [4]:
Tx = 30  #假设人类可读日期的最大长度; 如果我们得到更长的输入，我们将截断它
Ty = 10   #因为 “YYYY-MM-DD” 是 10 个字符长度)
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)
#X: X.shape = (m, Tx)  训练集中人类可读日期的处理版本, 其中每个字符都被它在 human_vocab 中映射该字符的索引替换。每个日期都使用特殊字符（）进一步填充。
#Y: Y.shape = (m, Ty)。 训练集中机器可读日期的处理版本, 其中每个字符都被它在machine_vocab中映射的索引替换。 
# Xoh:  Xoh.shape = (m, Tx, 37)    X 的 one-hot 版本, one-hot 中条目 “1” 的索引被映射到在human_vocab中对应字符。
# Yoh: Yoh.shape = (m, Tx, 11)。 这里11 因为有 11 字符 (’-’ 以及 0-9).
#                                  Y 的 one-hot 版本, one-hot 中条目 “1” 的索引被映射到由于machine_vocab 中对应字符。
print("Y.shape:", Y.shape)
print("X.shape:", X.shape)


index = 0
print("Source date:", dataset[index][0])  #Source date: 9 may 1998
print("Target date:", dataset[index][1])   #Target date: 1998-05-09



Y.shape:

(10000, 10)

X.shape:

(10000, 30)

Source date:

9 may 1998

Target date:

1998-05-09

In [5]:
# 带注意力的神经机器翻译

#模型中有两个单独的 LSTM。
# pre-attention Bi-LSTM  (a):一个是图片底部的那个是 Bi-LSTM（双向LSTM）在 Attention 前，我们叫做 pre-attention Bi-LSTM。pre-attention Bi-LSTM 经历 Tx 时间步；
# post-attention LSTM    (s): 图的顶部的 LSTM 在 Attention 后，我们叫做 post-attention LSTM。post-attention LSTM 经历 Ty 时间步；

# context_⟨t⟩ =  ∑_t ′ α ⟨t,t ′⟩*a ⟨t ′⟩
# context<t> = ∑ a*权重


# 将共享层定义为全局变量 
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') #  softmax(axis = 1)
dotor = Dot(axes = 1)


n_a = 32    # pre-attention Bi-LSTM   双向 a --Bi-LSTM的输出隐藏状态  维度 (m, Tx, 2*n_a)
n_s = 64    # post-attention LSTM   s_prev -- (post-attention) LSTM的前一个隐藏状态, 维度(m, n_s)
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [7]:
# GRADED FUNCTION: one_step_attention
def one_step_attention(a, s_prev):
    """
    执行一步 attention: 输出一个上下文向量，输出作为注意力权重的点积计算的上下文向量
    "alphas"  Bi-LSTM的 隐藏状态 "a"
    参数：
    a --  Bi-LSTM的输出隐藏状态 numpy-array 维度 (m, Tx, 2*n_a)
    s_prev -- (post-attention) LSTM的前一个隐藏状态, 维度(m, n_s)
    返回：
    context -- 上下文向量, 下一个(post-attetion) LSTM 单元的输入
    """
    # 使用 repeator 重复 s_prev 维度 (m, Tx, n_s) 这样就可以将它与所有隐藏状态"a" 连接起来
    s_prev = repeator(s_prev)
    # 使用 concatenator 在最后一个轴上连接 a 和 s_prev 
    concat = concatenator([a, s_prev])
    # 使用 densor1 传入参数 concat, 通过一个小的全连接神经网络来计算“中间能量”变量 e。
    e = densor1(concat)
    # 使用 densor2 传入参数 e , 通过一个小的全连接神经网络来计算“能量”变量 energies。
    energies = densor2(e)
    # 使用 activator 传入参数 "energies" 计算注意力权重 "alphas" 
    alphas = activator(energies)
    # 使用 dotor 传入参数 "alphas" 和 "a" 计算下一个（(post-attention) LSTM 单元的上下文向量line)
    context = dotor([alphas, a])
    return context




# GRADED FUNCTION: model
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    参数:
    Tx -- 输入序列的长度  30
    Ty -- 输出序列的长度  10
    n_a -- Bi-LSTM的隐藏状态大小  32
    n_s -- post-attention LSTM的隐藏状态大小  64
    human_vocab_size -- python字典 "human_vocab" 的大小
    machine_vocab_size -- python字典 "machine_vocab" 的大小
    返回：
    model -- Keras 模型实例
    """
    # 定义模型的输入，维度 (Tx,)
    # 定义 s0 和 c0, 初始化解码器 LSTM 的隐藏状态，维度 (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # 初始化一个空的输出列表
    outputs = []
    
    # 第一步：定义 pre-attention Bi-LSTM。 记得使用 return_sequences=True
    a = Bidirectional(LSTM(n_a, return_sequences=True), input_shape=(m, Tx, n_a * 2))(X)
    
    # 第二步：迭代 Ty 步
    for t in range(Ty):
        # 第二步.A: 执行一步注意机制，得到在 t 步的上下文向量 
        context = one_step_attention(a, s)
        # 第二步.B: 使用 post-attention LSTM 单元得到新的 "context" 
        # 别忘了使用： initial_state = [hidden state, cell state]
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        # 第二步.C: 使用全连接层处理post-attention LSTM 的隐藏状态输出
        out = output_layer(s)
        # 第二步.D: 追加 "out" 到 "outputs" 列表 
        outputs.append(out)
    
    # 第三步：创建模型实例，获取三个输入并返回输出列表
    model = Model(inputs=[X, s0, c0], outputs=outputs)
    return model

model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [8]:
model.summary()

__________________________________________________________________________________________________

Layer (type)                    Output Shape         Param #     Connected to                     

input_2 (InputLayer)            (None, 30, 37)       0                                            

__________________________________________________________________________________________________

s0 (InputLayer)                 (None, 64)           0                                            

...

                                                                 concatenate_1[12][0]             

                                                                 concatenate_1[13][0]             

                                                                 concatenate_1[14][0]             

                                                                 concatenate_1[15][0]             

                                                                 concatenate_1[16][0]             

In [9]:
# 在Keras创建模型后，需要编译它并定义模型使用的损失, 优化和评估指标。
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#已经有包含训练样例的 X， 维度 (m=10000,Tx=30) (m = 10000, T_x = 30)
# 需要创建用0初始化的 s0 和 c0，用于初始的 post_activation_LSTM_cell。j]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

model.fit([Xoh, s0, c0], outputs, epochs=1, batch_size=100)


Instructions for updating:
Use tf.cast instead.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Epoch 1/1

  100/10000 [..............................]

 - ETA: 20:56 - loss: 24.0487 - dense_3_loss: 2.3912 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.2100 - dense_3_acc_2: 0.1400 - dense_3_acc_3: 0.0800 - dense_3_acc_4: 0.0000e+00 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0600 - dense_3_acc_7: 0.0000e+00 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.1300

...

 9900/10000 [============================>.]

 - ETA: 0s - loss: 17.1318 - dense_3_loss: 2.6047 - dense_3_acc: 0.3976 - dense_3_acc_1: 0.6584 - dense_3_acc_2: 0.2937 - dense_3_acc_3: 0.0667 - dense_3_acc_4: 0.8610 - dense_3_acc_5: 0.3775 - dense_3_acc_6: 0.0621 - dense_3_acc_7: 0.9183 - dense_3_acc_8: 0.2047 - dense_3_acc_9: 0.0838



10000/10000 [==============================]

 - 28s 3ms/step - loss: 17.0659 - dense_3_loss: 2.6012 - dense_3_acc: 0.4022 - dense_3_acc_1: 0.6611 - dense_3_acc_2: 0.2955 - dense_3_acc_3: 0.0673 - dense_3_acc_4: 0.8624 - dense_3_acc_5: 0.3819 - dense_3_acc_6: 0.0643 - dense_3_acc_7: 0.9191 - dense_3_acc_8: 0.2068 - dense_3_acc_9: 0.0849


In [10]:
model.load_weights(r'C:\workspace\python-work\python 3\cs230_deeplearning_implement\ec_code\class5\week3\MachineTranslation\models\model.h5')

#测试结果
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001']
for example in EXAMPLES:
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = np.expand_dims(source, axis=0)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output))

source:

3 May 1979

output:

1979-05-03

source:

...

output:

2001-03-03

source:

March 3rd 2001

output: